In [1]:
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from skimage import filters

import tensorflow as tf
from tensorflow.keras import Model, Sequential 
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Conv2D, MultiHeadAttention, AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D , Conv3D, Layer, MaxPooling2D, Dropout, Flatten, Dense, GRU, ConvLSTM2D, Input, BatchNormalization, TimeDistributed, MaxPooling3D, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from random import shuffle
from tensorflow.keras.regularizers import l2

# from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
from keras.models import Model, load_model


# import cv2
# import imageio
# import cv2

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
# import IPython
# from six.moves import urllib
print(tf.__version__)
print(keras.__version__)
import pydicom
import re
import math
import random
# import bisect
np.random.seed(1234)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import warnings
warnings.filterwarnings(action='once')

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 1024
IMG_SIZE = 512
BATCH_SIZE = 4
EPOCHS = 2
NUM_SCANS = 8
NUM_CHANNELS = 1
INPUT_DIM = 256

keras.utils.set_random_seed(1)
random.seed(1)
np.random.seed(1)

2.8.1
2.8.0
Num GPUs Available:  1


In [2]:
test = pd.read_csv('test_ids.csv')

/tmp/ipykernel_3347949/3022398301.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_ids.csv')


In [3]:
fold = 0
test = pd.read_csv('test_ids.csv')
test = test[test.ycoord != 'ERROR']
test.ycoord = test.ycoord.replace('True', '1.0').astype('float')
test = test.drop_duplicates('StudyInstanceUID')
print(len(test))
directory = '/home/shared/test/coat_np_0' + str(fold) + '/'

lisdir = os.listdir(directory)
print('hello?')
print(len(lisdir))

files = pd.DataFrame({'file_name':lisdir})
files['StudyInstanceUID'] = files['file_name'].str.replace('.npy','')
files = pd.merge(files, test)
files = files.drop(columns=['SeriesInstanceUID', 'SOPInstanceUID','ycoord'])

650
hello?
650


/tmp/ipykernel_3347949/2467093630.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_ids.csv')
/tmp/ipykernel_3347949/2467093630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.ycoord = test.ycoord.replace('True', '1.0').astype('float')
/tmp/ipykernel_3347949/2467093630.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  files['StudyInstanceUID'] = files['file_name'].str.replace('.npy','')


In [4]:
num_features = 64
MAX_SEQ_LENGTH = 200

class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, study_ids, num_features, directory,
                 seq_length, return_type = 'both', to_fit=True, batch_size=32, 
                 shuffle=True, full_set = None, random=False, set_type='test',
                 feat_type = 'feats', oversample=True, crop=False ):

        self.list_IDs = list_IDs
        self.study_ids = study_ids
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_features = num_features
        self.seq_length = seq_length
        self.full_set = full_set
        self.return_type = return_type
        self.random = random
        warnings.filterwarnings(action='ignore')
        self.directory = directory
        self.set_type = set_type
        self.feat_type = feat_type
        self.oversample=oversample
        self.crop= crop
        
        if self.set_type == 'val':
            sample_pos = self.full_set[self.full_set.negative_exam_for_pe == False]
            sample_neg = self.full_set[self.full_set.negative_exam_for_pe == True]
            self.study_ids = pd.concat([sample_pos, sample_neg.sample(n=len(sample_pos))]).sample(frac = 1).reset_index(drop=True)
            self.list_IDs = np.arange(0, len(self.study_ids))
        self.on_epoch_end()
            
        
    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
#         print('starting')
        indexes = self.list_IDs[index * self.batch_size:((index+1) * self.batch_size)]
        x_feats = np.zeros([self.batch_size, self.seq_length, self.num_features])
        y_seq = np.zeros([self.batch_size, self.seq_length, 1])
        y_tot = np.zeros([self.batch_size, 1])
        for i in range(0,self.batch_size):
            x, y = self._get_scan_data(self.study_ids.iloc[indexes[i]].StudyInstanceUID)
            x_feats[i] = x
            y_seq[i] = y[0]
            y_tot[i] = y[1]
            
        if self.to_fit:
            if self.return_type =='both':
                y = [np.array(y_seq), y_tot]
            elif self.return_type == 'seq':
                y = y_seq
            elif self.return_type == 'tot':
                y = y_tot
            else:
                print('valid return types are both, seq and tot')
                return False
            x = x_feats
            return x, y
        else:
            return (X)
        
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        if self.set_type == 'train':
            sample_pos = self.full_set[self.full_set.negative_exam_for_pe == False]
            sample_neg = self.full_set[self.full_set.negative_exam_for_pe == True]
            if self.oversample == False:
                self.study_ids = pd.concat([sample_pos, sample_neg.sample(n=len(sample_pos))]).sample(frac=1).reset_index(drop=True)
            else:
                self.study_ids = pd.concat([sample_pos.sample(n=len(sample_neg), replace=True), sample_neg]).sample(frac = 1).reset_index(drop=True)
            self.list_IDs = np.arange(0, len(self.study_ids))
        if self.shuffle == True:
            np.random.shuffle(self.list_IDs)
            
    def _get_scan_data(self, study_id):
        scan = pd.DataFrame(np.load(self.directory +study_id + '.npy', allow_pickle=True).tolist())
        scan.ycoord = scan.ycoord.replace('True', '1.0').astype('float')
        scan = scan.sort_values(by=['ycoord'], ascending=False).reset_index(drop=True)
        
        features = scan.features.tolist()
        seq = scan.pe_present_on_image.tolist()
        seq = np.reshape(seq, [len(seq),1])
        tot = self.study_ids[self.study_ids.StudyInstanceUID == study_id].negative_exam_for_pe.iloc[0]
        
        if self.set_type == 'train' and self.random == True:
            new_len = int(len(scan) * (random.random()*0.25 +0.9))
            inst = np.round(np.arange(0,new_len)/new_len*len(features)).astype(int)
            features = (np.array(features)[inst]).tolist()
            seq = seq[inst]   
            
            new_len = int(len(features)*(1 - np.random.random() * 0.05))
            new_start = int(np.random.random() * (len(features)- new_len))
            features = features[new_start:new_start+new_len]
            seq = seq[new_start:new_start+new_len]        
        
        
        if len(features)>=self.seq_length:
            if self.crop==False:
                inst = np.round(np.arange(0,self.seq_length)/self.seq_length*len(features)).astype(int)
                xs = (np.array(features)[inst]).tolist()
                ys = seq[inst]
            if self.crop==True:
                xs = (np.array(features)[:self.seq_length]).tolist()
                ys = seq[:self.seq_length]
        else:
            xs = np.zeros([self.seq_length, self.num_features])
            ys = np.zeros([self.seq_length, 1])

            start = int(np.floor((self.seq_length - len(features))/2))
            end = start + len(features)
            xs[start:end] = features
            ys[start:end] = seq

        ys = ys.tolist()
                    
        return (xs, [ys, tot])

    
def get_generators(df, fold, batch_size, out, feat, random, seq_len=200, crop=True, np_dir='/home/shared/coat_np_0', osa=True, test_only=False):

    dire = np_dir + str(fold) + '/'

    if test_only:
        return DataGenerator(np.arange(0, len(df)),
                                    df, 
                                    64,
                                    dire,
                                    seq_len,
                                    batch_size=batch_size,
                                    set_type = 'test',
                                    return_type=out,
                                    feat_type=feat,
                                    crop=crop,
                                    shuffle=False)
    
    test_df = df[df.fold == fold]
    train_df = df[df.fold != fold]
        
    print(train_df.fold.unique())
    print(test_df.fold.unique())
    test_generator = DataGenerator(np.arange(0, len(test_df)),
                                    test_df, 
                                    64,
                                    dire,
                                    seq_len,
                                    batch_size=batch_size,
                                    set_type = 'test',
                                    return_type=out,
                                    feat_type=feat,
                                    crop=crop)
    
    val_generator = DataGenerator(np.arange(0, len(test_df)),
                                  test_df, 
                                  64,
                                  dire,
                                  seq_len,
                                  batch_size=batch_size,
                                  set_type ='val',
                                  full_set=test_df,
                                  return_type=out,
                                  feat_type=feat,
                                  crop=crop)
    
    train_generator = DataGenerator(np.arange(0, len(train_df)),
                                  train_df, 
                                  64,
                                  dire,
                                  seq_len,
                                  batch_size=batch_size,
                                  set_type ='train',
                                  full_set = train_df,
                                  random = random,
                                  return_type=out,
                                  feat_type=feat,
                                  crop=crop,
                                  oversample=osa)
    
    return train_generator, val_generator, test_generator

In [5]:
def get_attn_lstm(seq_len, noise=0.001):
        num_heads = 1
        key_dim = 8
        inputs = keras.Input(shape=(seq_len, 64))
        x = layers.BatchNormalization()(inputs)
        x = layers.GaussianNoise(noise)(x)
        x = layers.TimeDistributed(layers.Dropout(0.3))(x)
        attn = layers.MultiHeadAttention(num_heads=num_heads,
                                                     key_dim=key_dim, 
                                                     dropout=0.4
                                                    )(x, x)
        attn = layers.LayerNormalization()(attn)
        x = layers.Add()((x,attn))
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.Bidirectional(layers.LSTM(units=8, 
                                             return_sequences=True,
                                             dropout=0.3, 
                                             recurrent_regularizer=keras.regularizers.L2(0.01),
                                            ))(x)
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.LSTM(units=8, return_sequences=True,
                        dropout=0.3, 
#                         bias_regularizer=keras.regularizers.L2(0.005)
                       )(x)
        x = layers.TimeDistributed(layers.Dropout(0.4))(x)
        x = layers.Bidirectional(layers.LSTM(units=8,
                                             return_sequences=False, 
                                             dropout=0.3,
#                                              bias_regularizer=keras.regularizers.L2(0.005),
                                            ))(x)
        x = layers.Dropout(0.4)(x)
        stack_outputs = layers.Dense(1, activation='sigmoid', name='tot_out')(x)

        model = keras.models.Model(inputs=inputs, outputs=stack_outputs)
        

        return model

In [177]:
class TurkuGenerator(keras.utils.Sequence):
    def __init__(
        self,
        df: pd.DataFrame,
#         encoding_dict: Dict[str, np.ndarray],
#         filename_col: str,
#         label_col: str,
#         dataset_label_col: str,
        batch_size: int,
        encoding_dim: int,
        directory: str,
        num_slices: int = 96,
        shuffle: bool = True,
    ):
        self.df = df.copy()
        self.series_instance_uids = self.df["StudyInstanceUID"].unique()
        self.count_no_aug_series = len(self.series_instance_uids)
#         self.encoding_dict = encoding_dict
#         self.filename_col = filename_col
#         self.label_col = label_col
#         self.dataset_label_col = dataset_label_col
        self.batch_size = batch_size
        self.encoding_dim = encoding_dim
        self.num_slices = num_slices
        self.shuffle = shuffle
        self.directory = directory

        self.on_epoch_end()

    def __len__(self) -> int:
        return int(np.ceil(self.count_no_aug_series / self.batch_size))

    def __getitem__(self, index: int):
        offset = index * self.batch_size
        indexes = self.indexes[offset : min(offset + self.batch_size, self.df.shape[0])]

        UIDs = [self.series_instance_uids[k] for k in indexes]

        X, ys = self._prepare_batch(UIDs)

        return X, ys

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.series_instance_uids))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _prepare_batch(self, UIDs):
        batch_size = len(UIDs)
        X = np.zeros((batch_size, self.num_slices, self.encoding_dim), dtype=np.float32)
        y = np.zeros((batch_size, self.num_slices, 1), dtype=np.int32)
        y_stack = np.zeros((batch_size, 1), dtype=np.int32)

        for i, uid in enumerate(UIDs):

            scan = pd.DataFrame(np.load(self.directory + uid + '.npy', allow_pickle=True).tolist())
            scan.ycoord = scan.ycoord.replace('True', '1.0').astype('float')
            scan = scan.sort_values(by=['ycoord'], ascending=True).reset_index(drop=True)

            features = scan.features.tolist()
            seq = scan.pe_present_on_image.tolist()
            
            if len(features) > self.num_slices:
                X[i] = features[:self.num_slices]
                y[i,:,0] = seq[:self.num_slices]
            else:
                X[i, :len(features), :] = features
                y[i, :len(seq), 0] = seq
#             y_stack[i,0] = scan.negative_exam_for_pe.iloc[0]
            y_stack[i,0] = 0 
            
        return X, [y, y_stack]


def get_turku_generators(df, fold, batch_size, np_dir='/home/shared/nps/turku_0', test_only=False):
    directory = np_dir + str(fold) + '/'

    if test_only == True:
        test_generator = TurkuGenerator(df, 
                                       batch_size,
                                       64,
                                       directory,
                                       shuffle=False)
        return test_generator
    
    test_df = df[df.fold == fold]
    train_df = df[df.fold != fold]
    
    train_pos = train_df[train_df.negative_exam_for_pe == False]
    train_neg = train_df[train_df.negative_exam_for_pe == True]

    val_pos = test_df[test_df.negative_exam_for_pe == False]
    val_neg = test_df[test_df.negative_exam_for_pe == True]

    train = pd.concat([train_pos, train_neg.sample(n=len(train_pos))]).sample(frac = 1).reset_index(drop=True)
    val = pd.concat([val_pos, val_neg.sample(n=len(val_pos))]).sample(frac = 1).reset_index(drop=True)
    
#     print(train_df.fold.unique())
#     print(test_df.fold.unique())

    test_generator = TurkuGenerator(test_df,
                                    batch_size,
                                    64,
                                    directory)
    
    val_generator = TurkuGenerator(val,
                                    batch_size,
                                    64,
                                    directory)
    
    train_generator = TurkuGenerator(train,
                                    batch_size,
                                    64,
                                    directory)
    
    return train_generator, val_generator, test_generator

In [126]:
def get_tukru_model() -> keras.models.Model:
    """Get a custom LSTM based model for classifying a set of slice encodings.
    The model predicts presence of PE at both slice and stack level.
    """
    tf.keras.backend.clear_session()

    slice_stack = keras.Input(shape=(NUM_SLICES, ENCODING_DIM))

    x = layers.BatchNormalization()(slice_stack)
    x = layers.Bidirectional(layers.LSTM(units=64, return_sequences=True))(x)
    x = layers.TimeDistributed(layers.Dense(512, activation="relu"))(x)
    x = layers.TimeDistributed(layers.Dense(128, activation="relu"))(x)
    slice_outputs = layers.TimeDistributed(layers.Dense(1, activation="sigmoid"), name="slices")(x)

    reshaped_features = layers.Reshape((NUM_SLICES,))(slice_outputs)
    reshaped_features = layers.Dropout(0.05)(reshaped_features)
    stack_output = layers.Dense(1, activation="sigmoid", name="sequence")(reshaped_features)

    model = keras.models.Model(inputs=slice_stack, outputs=[slice_outputs, stack_output])

    return model


In [ ]:
seq_len=208
tot = 0
caught = 0

no = 0
nope = 0

test = files.sort_values('StudyInstanceUID')
test['negative_exam_for_pe'] = 0
for fold in range(0,1):

    model = get_attn_lstm(seq_len)

    np_dir = '/home/shared/test/coat_0'

    batch_size = 1
    test_gen = get_generators(test, fold, 
                                   batch_size, 
                                   'tot', 
                                   'feats', 
                                   True, 
                                   seq_len=seq_len, 
                                   np_dir=np_dir, 
                                   crop=False,
                                   osa=False,
                                   test_only=True)
    
    t_len = test_gen.__len__()
    x_test = np.zeros([t_len,208,num_features])
    y_test = np.zeros([t_len,1])
    for i in range(0,t_len):
        x_test[i],y= test_gen.__getitem__(i)
        y_test[i] = y
                
    lr = 0.001
    opt = keras.optimizers.Nadam(learning_rate=lr)
    model.compile(
            optimizer=opt,
            loss=keras.losses.binary_crossentropy,
            metrics=["accuracy",]
        )

    checkpoint_filepath = '/home/shared/model_checkpoint_paige/scan/reg5-kfold_0'+str(fold)
    model.load_weights(checkpoint_filepath)
    
    y_pred = model.predict(x_test, batch_size=32)
    test['fold_0'+str(fold)] = y_pred

In [180]:
test = files.sort_values('StudyInstanceUID')
test['negative_exam_for_pe'] = 0

NUM_SLICES=96
ENCODING_DIM = 64
for fold in range(0,10):

    model = get_tukru_model()
    opt = keras.optimizers.Nadam(learning_rate=0.001)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

    np_dir = '/home/shared/test/turku_0'

    batch_size = 1
    test_gen = get_turku_generators(test,fold,1,np_dir=np_dir,test_only=True)    
    
    t_len = test_gen.__len__()
    x_test = np.zeros([t_len,NUM_SLICES,ENCODING_DIM])
    y_test = np.zeros([t_len,1])
    for i in range(0,t_len):
        x_test[i],y= test_gen.__getitem__(i)
        y_test[i] = y[1]
                

    checkpoint_filepath = '/home/shared/model_checkpoint_paige/scan/turku_fold_0'+str(fold)
    model.load_weights(checkpoint_filepath)
    
    y_pred = model.predict(x_test, batch_size=32)
    test['fold_0'+str(fold)] = y_pred[1]




In [203]:
ans['avg'] = 0
for fold in range(0,10): 
    ans['avg'] += ans['fold_0'+str(fold)]/10

In [201]:
for fold in range(0,10):
    ans['fold_0'+str(fold)] = 1-ans['fold_0'+str(fold)]

In [204]:
ans

,Unnamed: 0,StudyInstanceUID,fold_00,fold_01,fold_02,fold_03,fold_04,fold_05,fold_06,fold_07,fold_08,fold_09,avg
0,362,00268ff88746,0.516448,0.101103,0.485995,0.091825,0.291079,0.127315,0.232077,0.410579,0.382904,0.142558,0.278188
1,313,00c53115a9fa,0.344861,0.342433,0.693427,0.466849,0.327386,0.290157,0.232099,0.068455,0.140639,0.076027,0.298233
2,514,00e7015490cb,0.193868,0.071207,0.059365,0.076562,0.030576,0.073124,0.055059,0.068053,0.103279,0.112114,0.084321
3,8,017d4bd72a02,0.983563,0.987083,0.991697,0.916703,0.994702,0.994667,0.992388,0.992160,0.990097,0.994776,0.983783
4,41,0185602e0766,0.082452,0.097081,0.055920,0.077840,0.033091,0.124356,0.054448,0.057666,0.064574,0.056883,0.070431
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,27,fda6fb996297,0.044739,0.072529,0.093087,0.075421,0.030814,0.023850,0.062034,0.058166,0.078919,0.059445,0.059900
646,476,fdbbd076dcdb,0.184319,0.098847,0.418313,0.206726,0.237240,0.362425,0.111578,0.270715,0.154030,0.159400,0.220359
647,336,fde9ecc1c2b5,0.080061,0.118764,0.094568,0.076470,0.034160,0.024418,0.090630,0.062398,0.109138,0.059092,0.074970
648,517,fe449a869d7a,0.290019,0.119091,0.094762,0.075550,0.172891,0.028744,0.057242,0.058463,0.120106,0.091727,0.110860


In [185]:
(np.abs(test[:200].fold_00.round() - test[:200].fold_06.round())).sum()

23.0

In [205]:
ans = ans.drop(columns=['Unnamed: 0'])

In [206]:
ans.to_csv('test_preds.csv')

In [74]:
ip_ans['StudyInstanceUID'] = ip_ans.id

In [75]:
comp = pd.merge(ip_ans, test, on='StudyInstanceUID')

In [76]:
ip_ans['StudyInstanceUID'] = ip_ans.id.str.replace('_negative_exam_for_pe','')

In [77]:
comp = pd.merge(ip_ans, test, on='StudyInstanceUID')

In [79]:
comp = comp.drop(columns=['StudyInstanceUID'])

In [94]:
comp['sum'] = 0
for fold in range(0,10): 
    comp['sum'] = comp['sum'] + comp['fold_0' +str(fold)].round()

In [210]:
test = pd.read_csv('test_preds.csv')
bench = pd.read_csv('benchmark_preds.csv')

In [215]:
test.avg.mean()

0.3738914155132462

In [217]:
bench.fold_00.mean()

0.5037020651384616

In [225]:
np.sum(bench.fold_00.round()), np.sum(test.fold_00.round())

(209.0, 194.0)

In [221]:
np.mean(np.abs(test.avg.round() - test.fold_00.round()))

0.09846153846153846

In [249]:
test = pd.DataFrame(columns=['Accuracy','Sensitivity','Specificity','PPV',
                             'NPV','F1Score','ROC_AUC'])

test = test.append([{'Accuracy': 89.23,
    'Sensitivity': 82.29,
    'Specificity': 92.14,
    'PPV': 81.44,
    'NPV': 92.54,
    'F1Score': 0.8186528497409326,
    'ROC_AUC': 0.9212609170305677},
    {'Accuracy': 86.46,
    'Sensitivity': 82.81,
    'Specificity': 87.99,
    'PPV': 74.3,
    'NPV': 92.43,
    'F1Score': 0.7832512315270936,
    'ROC_AUC': 0.9276519286754004},
    {'Accuracy': 87.69,
    'Sensitivity': 83.85,
    'Specificity': 89.3,
    'PPV': 76.67,
    'NPV': 92.95,
    'F1Score': 0.8009950248756219,
    'ROC_AUC': 0.9254798944687045},
    {'Accuracy': 87.23,
    'Sensitivity': 78.65,
    'Specificity': 90.83,
    'PPV': 78.24,
    'NPV': 91.03,
    'F1Score': 0.7844155844155843,
    'ROC_AUC': 0.9140170123726347},
    {'Accuracy': 87.38,
    'Sensitivity': 79.69,
    'Specificity': 90.61,
    'PPV': 78.06,
    'NPV': 91.41,
    'F1Score': 0.788659793814433,
    'ROC_AUC': 0.9151314592430859},
    {'Accuracy': 88.92,
    'Sensitivity': 81.25,
    'Specificity': 92.14,
    'PPV': 81.25,
    'NPV': 92.14,
    'F1Score': 0.8125,
    'ROC_AUC': 0.9324053857350799,},
    {'Accuracy': 87.08,
    'Sensitivity': 81.25,
    'Specificity': 89.52,
    'PPV': 76.47,
    'NPV': 91.93,
    'F1Score': 0.787878787878788,
    'ROC_AUC': 0.9285161935953421},
    {'Accuracy': 86.92,
    'Sensitivity': 77.6,
    'Specificity': 90.83,
    'PPV': 78.01,
    'NPV': 90.63,
    'F1Score': 0.7780678851174934,
    'ROC_AUC': 0.9140738719068414},
    {'Accuracy': 87.85,
    'Sensitivity': 81.77,
    'Specificity': 90.39,
    'PPV': 78.11,
    'NPV': 92.2,
    'F1Score': 0.7989821882951654,
    'ROC_AUC': 0.9159502365356623},
    {'Accuracy': 87.69,
    'Sensitivity': 79.69,
    'Specificity': 91.05,
    'PPV': 78.87,
    'NPV': 91.45,
    'F1Score': 0.7927461139896373,
    'ROC_AUC': 0.9081377365356624}])

# 'Accuracy': 89.69,
# 'Sensitivity': 80.73,
# 'Specificity': 93.45,
# 'PPV': 83.78,
# 'NPV': 92.04,
# 'F1Score': 0.8222811671087533,
# 'ROC_AUC': 0.9374545123726347




In [231]:
bench = pd.DataFrame(columns=['Accuracy','Sensitivity','Specificity','PPV',
                             'NPV','F1Score','ROC_AUC'])

bench = bench.append([{'Accuracy': 84.15,
    'Sensitivity': 77.6,
    'Specificity': 86.9,
    'PPV': 71.29,
    'NPV': 90.25,
    'F1Score': 0.743142144638404,
    'ROC_AUC': 0.8819141193595342},
    {'Accuracy': 86.31,
    'Sensitivity': 71.88,
    'Specificity': 92.36,
    'PPV': 79.77,
    'NPV': 88.68,
    'F1Score': 0.7561643835616437,
    'ROC_AUC': 0.8922056950509462},
    {'Accuracy': 84.77,
    'Sensitivity': 76.04,
    'Specificity': 88.43,
    'PPV': 73.37,
    'NPV': 89.8,
    'F1Score': 0.7468030690537084,
    'ROC_AUC': 0.8944232168850071},
    {'Accuracy': 87.69,
    'Sensitivity': 76.56,
    'Specificity': 92.36,
    'PPV': 80.77,
    'NPV': 90.38,
    'F1Score': 0.786096256684492,
    'ROC_AUC': 0.8911822234352257},
    {'Accuracy': 87.85,
    'Sensitivity': 74.48,
    'Specificity': 93.45,
    'PPV': 82.66,
    'NPV': 89.73,
    'F1Score': 0.7835616438356164,
    'ROC_AUC': 0.9017012372634643},
    {'Accuracy': 84.46,
    'Sensitivity': 77.08,
    'Specificity': 87.55,
    'PPV': 72.2,
    'NPV': 90.11,
    'F1Score': 0.7455919395465995,
    'ROC_AUC': 0.9017467248908295},
    {'Accuracy': 85.54,
    'Sensitivity': 68.23,
    'Specificity': 92.79,
    'PPV': 79.88,
    'NPV': 87.45,
    'F1Score': 0.7359550561797754,
    'ROC_AUC': 0.8890443049490538},
    {'Accuracy': 85.23,
    'Sensitivity': 78.65,
    'Specificity': 87.99,
    'PPV': 73.3,
    'NPV': 90.77,
    'F1Score': 0.7587939698492463,
    'ROC_AUC': 0.9007118813682679},
    {'Accuracy': 85.54,
    'Sensitivity': 72.4,
    'Specificity': 91.05,
    'PPV': 77.22,
    'NPV': 88.72,
    'F1Score': 0.7473118279569892,
    'ROC_AUC': 0.8927970342066959},
    {'Accuracy': 85.08,
    'Sensitivity': 74.48,
    'Specificity': 89.52,
    'PPV': 74.87,
    'NPV': 89.32,
    'F1Score': 0.7467362924281983,
    'ROC_AUC': 0.8900791484716157}])
# Evaluation of avg
# 'Accuracy': 86.46,
# 'Sensitivity': 76.56,
# 'Specificity': 90.61,
# 'PPV': 77.37,
# 'NPV': 90.22,
# 'F1Score': 0.769633507853403
# 'ROC_AUC': 0.905510826055313

In [269]:
test

,Accuracy,Sensitivity,Specificity,PPV,NPV,F1Score,ROC_AUC
0,89.23,82.29,92.14,81.44,92.54,0.818653,0.921261
1,86.46,82.81,87.99,74.30,92.43,0.783251,0.927652
2,87.69,83.85,89.30,76.67,92.95,0.800995,0.925480
3,87.23,78.65,90.83,78.24,91.03,0.784416,0.914017
4,87.38,79.69,90.61,78.06,91.41,0.788660,0.915131
5,88.92,81.25,92.14,81.25,92.14,0.812500,0.932405
6,87.08,81.25,89.52,76.47,91.93,0.787879,0.928516
7,86.92,77.60,90.83,78.01,90.63,0.778068,0.914074
8,87.85,81.77,90.39,78.11,92.20,0.798982,0.915950
9,87.69,79.69,91.05,78.87,91.45,0.792746,0.908138


In [267]:
from scipy.stats import ranksums
for col in bench.columns:
    print(col)
    test[col] = test[col].astype('double')
    bench[col] = bench[col].astype('double')
    print('mean test: {:.3f} mean bench: {:.3f} p-value {:.8f}'.format(test[col].mean(), bench[col].mean(), ranksums(test[col][2], bench[col][2])[1]))
    print(ranksums(test[col],bench[col]))
    print()

Accuracy
mean test: 87.645 mean bench: 85.662 p-value 0.31731051
RanksumsResult(statistic=2.7591406529673588, pvalue=0.00579535854433471)

Sensitivity
mean test: 80.885 mean bench: 74.740 p-value 0.31731051
RanksumsResult(statistic=3.6284589408885815, pvalue=0.00028511808363161265)

Specificity
mean test: 90.480 mean bench: 90.240 p-value 0.31731051
RanksumsResult(statistic=-0.11338934190276817, pvalue=0.9097218891455553)

PPV
mean test: 78.142 mean bench: 76.533 p-value 0.31731051
RanksumsResult(statistic=0.9071147352221454, pvalue=0.3643461266335529)

NPV
mean test: 91.871 mean bench: 89.521 p-value 0.31731051
RanksumsResult(statistic=3.704051835490427, pvalue=0.00021218287122257823)

F1Score
mean test: 0.795 mean bench: 0.755 p-value 0.31731051
RanksumsResult(statistic=3.4016802570830453, pvalue=0.0006697294490218271)

ROC_AUC
mean test: 0.920 mean bench: 0.894 p-value 0.31731051
RanksumsResult(statistic=3.779644730092272, pvalue=0.00015705228423075119)

